# Getting started

First, we enable the cluster to scale up. Note that if you run an auto-scaling cluster,
Google will suspend your nodes. Make sure to have the experiment prepared before running the commands.

The following is assumed ready:
* GKE/Kubernetes cluster (see also `terraform/terraform_notebook.ipynb`)
    * 2 nodes pools (default for system & dependencies, experiment pool)
* Docker image (including dataset, to speed-up starting experiments).
    * Within a bash shell
        * Make sure to have the `requirements-cpu.txt` installed (or `requirements-gpu.txt (in a virtual venv/conda environment). You can run `pip3 install -r requirements-cpu.txt`
    * First run the extractor (locally) `python3 -m fltk extractor configs/example_cloud_experiment.json`
        *  This downloads datasets to be included in the docker image.
    * Build the container `DOCKER_BUILDKIT=1 docker build --platform linux/amd64 . --tag gcr.io/$PROJECT_ID/fltk`
    * Push to your gcr.io repository `docker push gcr.io/$PROJECT_ID/fltk`


With that setup, first set some variables used throughout the experiment.


In [10]:
##################
### CHANGE ME! ###
##################
PROJECT_ID="fltk-flint"
CLUSTER_NAME="fltk-testbed-cluster"
DEFAULT_POOL="default-node-pool"
EXPERIMENT_POOL="medium-fltk-pool-1"
REGION="us-central1-c"

# In case we do not yet have the credentials/kubeconfig
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

Fetching cluster endpoint and auth data.
kubeconfig entry generated for fltk-testbed-cluster.


Scale the default-node-pool up.

In [ ]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
     --num-nodes 1 --region $REGION --quiet

## Preparation
In case you have already tested something or ran another experiment, we have to remove the deployment of the Orchestrator. This will not delete any experiment data, as this persists on one of the ReadWriteMany PVCs.


Currently, the Orchestrator is deployed using a `Deployment` definition, a future version will replace this with a `Deployment` definition, to make this step unnecessary. For experiments this means the following:

1. A single deployment can exist at a single time in a single namespace. This includes 'completed' experiments.
2. For running batches of experiments, a BatchOrchestrator is provided.


ℹ️ This will not remove any data, but if your orchestrator is still/already running experiments, this will stop the deployment. Running training jobs will not be stopped, for this you can use `kubectl`. ConfigMaps created by the Orchestrator (to provide experiment configurations), will not be removed. See the commented code in the cell below.

In [ ]:
# If you want to delete all pytorch trainjobs, uncomment the command below.
# kubectl delete pytorchjobs.kubeflow.org --all --namespace test

# If you want to delete all existing configuration map objects in a namespace, run teh command below
# kubectl delete configmaps --all --namespace test

helm uninstall -n test flearner

## Install extractor

Deploy the TensorBoard service and persistent volumes, required for deployment of the orchestrator's chart.

In [2]:
helm upgrade --install -n test extractor ../charts/extractor -f ../charts/fltk-values.yaml \
    --set provider.projectName=$PROJECT_ID

W1010 15:27:40.226740   17542 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Release "extractor" has been upgraded. Happy Helming!
NAME: extractor
LAST DEPLOYED: Mon Oct 10 15:27:42 2022
NAMESPACE: test
STATUS: deployed
REVISION: 2
TEST SUITE: None
NOTES:
Get the FLTK extractors Tensorboard URL by running:

export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006


## Define experiment configuration files

Deployment of experiments is currently done through a Helm Deployment. A future release (™️) will rework this to a Job definition, as this allows to re-use the template more easily.


> The `EXPERIMENT_FILE` will contain the description of the experiments
> The `CLUSTER_CONFIG` will contain shared configurations for logging, Orchestrator configuration and replication information.

In [3]:
EXPERIMENT_FILE="../configs/distributed_tasks/example_arrival_config.json"
CLUSTER_CONFIG="../configs/example_cloud_experiment.json"

## Setup experiment variables
Next, we will deploy the experiments.


We provide a configuration file, `charts/fltk-values.yaml`, in here change the values under the `provider` block. Change `projectName` to your Google Cloud Project ID.

```yaml
provider:
    domain: gcr.io
    projectName: CHANGE_ME!
    imageName: fltk:latest
```

We use the `--set-file` flag for `helm`, as currently, Helm does not support using files outside of the chart root directory (in this case `charts/orchestrator`). Using `--set-file` we can dynamically provide these files. See also issue [here](https://github.com/helm/helm/issues/3276)


In [4]:
helm uninstall -n test experiment-orchestrator
helm install -n test experiment-orchestrator ../charts/orchestrator -f ../charts/fltk-values.yaml \
    --set-file orchestrator.experiment=$EXPERIMENT_FILE,orchestrator.configuration=$CLUSTER_CONFIG \
    --set provider.projectName=$PROJECT_ID

W1010 15:30:30.458714   17913 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: experiment-orchestrator: release: not found
W1010 15:30:31.215955   17930 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME: experiment-orchestrator
LAST DEPLOYED: Mon Oct 10 15:30:34 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You successfully launched an experiment configuration on your cluster in test.

N.B. Make sure to collect all data after completing your experiment!
N.B. Re-installing the orchestrator WILL RESULT IN DELETION OF ALL TRAINJOBS and PODS!


In [5]:
# To get logs from the orchestrator
kubectl logs -n test fl-server

W1010 15:41:37.210663   17960 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-10-2022 13:30:39 root         INFO     Loading file config/configuration.fltk.json
No argument path is provided.
10-10-2022 13:30:39 root         INFO     Starting in cluster mode.
10-10-2022 13:30:39 root         INFO     Starting with experiment replication: 0 with seed: 42
10-10-2022 13:30:39 root         INFO     Starting as Orchestrator
10-10-2022 13:30:39 root         INFO     Starting Orchestrator, initializing resources....
10-10-2022 13:30:39 root         INFO     Loading in cluster configuration file
10-10-2022 13:30:39 root         INFO     Pointing configuration to in cluster configuration.
10-10-2022 13:30:39 root         INFO     Starting cluster manager
10-10-2022 13:30:39 ClusterManager INFO     Spinning up cluster manag

10-10-2022 13:33:42 ResourceWatchDog INFO     Fetching pod information of cluster...
10-10-2022 13:33:45 root         INFO     Waiting for bbf0f3e8-4512-4d0e-98c1-a7630da04f50 to complete, 0 pending, 0 arrivals
10-10-2022 13:33:50 root         INFO     Waiting for bbf0f3e8-4512-4d0e-98c1-a7630da04f50 to complete, 0 pending, 0 arrivals
10-10-2022 13:33:52 ResourceWatchDog INFO     Fetching pod information of cluster...
10-10-2022 13:33:55 root         INFO     Waiting for bbf0f3e8-4512-4d0e-98c1-a7630da04f50 to complete, 0 pending, 0 arrivals
10-10-2022 13:34:00 root         INFO     Waiting for bbf0f3e8-4512-4d0e-98c1-a7630da04f50 to complete, 0 pending, 0 arrivals
10-10-2022 13:34:02 ResourceWatchDog INFO     Fetching pod information of cluster...
10-10-2022 13:34:05 root         INFO     Waiting for bbf0f3e8-4512-4d0e-98c1-a7630da04f50 to complete, 0 pending, 0 arrivals
10-10-2022 13:34:10 root         INFO     Waiting for bbf0f3e8-4512-4d0e-98c1-a7630da04f50 to complete, 0 pending, 

10-10-2022 13:37:41 root         INFO     Waiting for bbf0f3e8-4512-4d0e-98c1-a7630da04f50 to complete, 0 pending, 0 arrivals
10-10-2022 13:37:46 root         INFO     Waiting for bbf0f3e8-4512-4d0e-98c1-a7630da04f50 to complete, 0 pending, 0 arrivals
10-10-2022 13:37:46 ResourceWatchDog INFO     Fetching pod information of cluster...
10-10-2022 13:37:46 ResourceWatchDog INFO     Fetching pod information of cluster...
10-10-2022 13:37:51 root         INFO     Waiting for bbf0f3e8-4512-4d0e-98c1-a7630da04f50 to complete, 0 pending, 0 arrivals
10-10-2022 13:37:56 root         INFO     Waiting for bbf0f3e8-4512-4d0e-98c1-a7630da04f50 to complete, 0 pending, 0 arrivals
10-10-2022 13:37:56 ResourceWatchDog INFO     Fetching pod information of cluster...
10-10-2022 13:38:01 root         INFO     Waiting for bbf0f3e8-4512-4d0e-98c1-a7630da04f50 to complete, 0 pending, 0 arrivals
10-10-2022 13:38:06 root         INFO     Waiting for bbf0f3e8-4512-4d0e-98c1-a7630da04f50 to complete, 0 pending, 

In [7]:
# To get logs from learners (example)
# kubectl logs -n test trainjob-bbf0f3e8-4512-4d0e-98c1-a7630da04f50-master-0

# To get logs from learners (federated learning)
kubectl logs -n test trainjob-bbf0f3e8-4512-4d0e-98c1-a7630da04f50-worker-0

W1010 15:46:09.572857   18093 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Defaulted container "pytorch" out of: pytorch, init-pytorch (init)
10-10-2022 13:32:55 root         INFO     Loading file config/configuration.fltk.json
10-10-2022 13:32:55 root         INFO     Starting in client mode
10-10-2022 13:32:55 root         INFO     Starting with host=trainjob-bbf0f3e8-4512-4d0e-98c1-a7630da04f50-master-0 and port=23456
10-10-2022 13:32:55 root         INFO     Initializing backend for training process: gloo
10-10-2022 13:32:55 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 1
10-10-2022 13:32:55 torch.distributed.distributed_c10d INFO     Rank 1: Completed store-based barrier for key:store_based_barrier_key:1 with 2 nodes.
10-10-2022 13:32:55 root         INFO

## Copy experiment results from the extractor

Extractor holds the experiment results in the format that can be processedby TensorBoard.
In order to download it to the local machine, execute:

In [9]:
EXTRACTOR_POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")

kubectl cp -n test $EXTRACTOR_POD_NAME:/opt/federation-lab/logging ./logging

W1010 15:47:02.808234   18139 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
W1010 15:47:03.364402   18159 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
tar: Removing leading `/' from member names


# Cleanup

## Removing orchestrator

In [8]:
helm uninstall -n test experiment-orchestrator

W1010 15:46:49.401752   18113 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


## Removing extractor

IMPORTANT: Removing extractor chart will result in deleting the already collected experiment results, stored in the NFS!

In [11]:
helm uninstall extractor -n test

W1010 15:58:34.710213   23525 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "extractor" uninstalled


## Wrapping up

To scale down the cluster nodepools, run the cell below. This will scale the node pools down and remove all the experiments deployed (on the cluster).

1. Experiments cannot be restarted.
2. Experiment logs will not persist deletion.


In [12]:
# This will remove all information and logs as well.
kubectl delete pytorchjobs.kubeflow.org --all-namespaces --all

gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
    --num-nodes 0 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 0 --region $REGION --quiet

W1010 15:58:46.336970   23558 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
pytorchjob.kubeflow.org "trainjob-bbf0f3e8-4512-4d0e-98c1-a7630da04f50" deleted
Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/fltk-flint/zones/us-central1-c/clusters/fltk-testbed-cluster].
Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/fltk-flint/zones/us-central1-c/clusters/fltk-testbed-cluster].
